<a href="https://colab.research.google.com/github/naghmeh1976/Tek-Team-Relevant-First-Sentences-Extraction/blob/main/Relevant%26FirstSentences_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence-transformers

In [ ]:
!pip install nltk

In [ ]:
import requests
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize
from sentence_transformers import SentenceTransformer, util
import nltk
nltk.download('punkt')


In [ ]:
def get_relevant_sentences(url):
    # Fetch webpage content
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    # Extract text from the webpage
    paragraphs = soup.find_all('p')
    article_text = ' '.join([paragraph.text for paragraph in paragraphs])

    # Tokenize article into sentences
    sentences = sent_tokenize(article_text)

    # Load SentenceTransformer model
    model = SentenceTransformer("BAAI/bge-m3")

    # Encode sentences
    sentence_embeddings = model.encode(sentences, convert_to_tensor=True)

    # Compute similarity scores between sentences and the first sentence
    query_embedding = sentence_embeddings[0]
    similarities = util.pytorch_cos_sim(query_embedding, sentence_embeddings)[0]

    # Define similarity threshold
    similarity_threshold = 0.60  # Adjust as needed

    # Select all sentences above the similarity threshold
    relevant_sentences = [sentence for sentence, score in zip(sentences, similarities) if score.item() > similarity_threshold]

    relevant_sentences_joined = '\n'.join(relevant_sentences)

    return relevant_sentences_joined


In [ ]:
def extract_first_sentence(url):
    # Send HTTP GET request to the URL
    response = requests.get(url)

    # Parse HTML content using BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extract the first paragraph and its text
    first_paragraph = soup.find('p')
    first_sentence = sent_tokenize(first_paragraph.text)[0]

    return first_sentence

In [ ]:
input_url=input("Enter URL to get the most relevant sentences")
relevant = get_relevant_sentences(input_url)
first = extract_first_sentence(input_url)

print("Relevant Sentences:")
print(relevant)
print()
print("\nFirst sentence of the article:")
print(first)

Enter URL to get the most relevant sentenceshttps://www.theguardian.com/technology/2023/apr/27/elon-musks-statements-could-be-deepfakes-tesla-defence-lawyers-tell-court
Relevant Sentences:
Judge in Autopilot death case says defence argument ‘deeply troubling’ and wants Tesla CEO interviewed under oath on safety claims A California judge has tentatively ordered Elon Musk to be interviewed under oath about whether he made certain statements regarding the capabilities of Tesla’s Autopilot features after the company questioned the authenticity of the remarks, claiming Musk is a “target for deep fakes”.
The attorneys for Huang’s family sought to depose Musk regarding recorded statements from 2016 in which he allegedly said: “A Model S and Model X, at this point, can drive autonomously with greater safety than a person.
Right now.” Tesla, however, opposed the request in court filings, arguing that Musk, the Tesla CEO, cannot recall details about the statement and questioning the authenticity